## 目录

- [0.书本介绍](https://applenob.github.io/deep_learning_1#0.书本介绍)
- [1. Introduction](https://applenob.github.io/deep_learning_1#1.-Introduction)
- [2. Linear Algebra](https://applenob.github.io/deep_learning_2)
- [3. Probability and Information Theory](https://applenob.github.io/deep_learning_3)
- [4. Numerical Computation](https://applenob.github.io/deep_learning_4)
- [5. Machine Learning Basics](https://applenob.github.io/deep_learning_5)
- [6. Deep Feedforward Networks](https://applenob.github.io/deep_learning_6)
- [7. Regularization for Deep Learning](https://applenob.github.io/deep_learning_7)
- [8. Optimization for Training Deep Models](https://applenob.github.io/deep_learning_8)
- [9. Convolutional Networks](https://applenob.github.io/deep_learning_9)
- [10. Sequence Modeling: Recurrent and Recursive Nets](https://applenob.github.io/deep_learning_10)
- [11. Practical Methodology](https://applenob.github.io/deep_learning_11)
- [12. Application](https://applenob.github.io/deep_learning_12)
- [13. Linear Factor Models](https://applenob.github.io/deep_learning_13)
- [14. Autoencoders](https://applenob.github.io/deep_learning_14)
- [15. Representation Learning](https://applenob.github.io/deep_learning_15)
- [16. Structured Probabilistic Models for Deep Learning](https://applenob.github.io/deep_learning_16)
- [17. Monte Carlo Methods](https://applenob.github.io/deep_learning_17)
- [18. Confronting the Partition Function](https://applenob.github.io/deep_learning_18)
- [19. Approximate Inference](https://applenob.github.io/deep_learning_19)
- [20. Deep Generative Models](https://applenob.github.io/deep_learning_20)

***
***


## 19. Approximate Inference

**推断（Inference）**：我们通常使用**推断**这个术语来指代给定一些其他变量的情况下计算某些变量概率分布的过程。 在深度学习中，通常我们有一系列**可见变量v**和一系列**隐变量h**。 推断困难通常是指难以计算$p(h∣v)$或其期望。 推断在最大似然学习的任务中往往是必需的。 但是，大多数具有多层隐藏变量的图模型的后验分布都很难处理。 因此需要想办法来实现**近似推断**。

**面对有隐变量的概率模型的思路**：这种情况下，目标是极大化观测数据（不完全数据）v关于参数θ的对数似然函数，即最大化：$L(θ)=logp(v;θ)=log\sum_hp(v,h;θ)=log(\sum_zp(h|v;θ)p(v;θ))$。 或者在图模型中，为了使用最大似然估计，有：$logp(v)=E_{h\sim p(h|v)}[logp(h,v)-logp(h|v)]$。 总之，计算推断，也就是$p(h|v)$是很有必要的。

**把推断视作优化问题**：精确推断问题可以描述为一个优化问题，计算一个$logp(v;θ)$的下界**$L(v,θ,q)$**（这个下界被称为**证据下界，evidence lower bound，ELBO**，另一个常用名称是负变分自由能，negative variational free energy）：$L(v,θ,q)=log⁡p(v;θ)−D_{KL}(q(h∣v)‖p(h∣v;θ))$，其中q是关于h的一个任意概率分布。 可以化简成：$L(v,θ,q)=E_{h\sim q}[logp(h,v)]+H(q)$。 对任意分布q的选择来说，L提供了似然函数的一个下界。 越好地近似$p(h∣v)$的分布$q(h∣v)$，得到的下界就越紧。 **将推断问题看作是找一个分布q使得L最大的过程。** 

**EM（Expectation Maximization）算法**：本质上，EM并不是一个近似推断算法，而是一种能够学到近似后验的算法；但是这里EM也可以解决近似推断问题。 **E步**: 令$θ^{(0)}$表示在这一步开始时的参数值。 对索引为i的训练样本$v^{(i)}$，$q(h^{(i)}∣v)=p(h^{(i)}∣v^{(i)};θ^{(0)})$。 认为q是在当前参数$θ^{(0)}$下定义的函数，也就是如果我们改变θ，那么$p(h∣v;θ)$将会相应地变化，但是$q(h∣v)$还是不变并且等于$p(h∣v;θ^{(0)})$。
**M步**：使用选择的优化算法关于θ最大化：$∑_iL(v^{(i)},θ,q)$。
这可以被看作通过**坐标上升（coordinate ascending）算法**来最大化L。 在第一步中，我们更新分布q来最大化L，而在另一步中，我们更新θ来最大化L。

**最大后验推断（MAP）**：$h^∗=\underset{h}{argmax}p(h∣v)$。 令分布q满足一个Dirac分布：$q(h∣v)=δ(h−μ)$。 这也意味着可以通过μ来完全控制分布q。 转换成另一个优化问题：$μ^∗=\underset{μ}{argmax} logp(h=μ,v)$。 使用类似EM算法的学习算法，还是轮流迭代两步，一步是用MAP推断估计出$h^*$，另一步是更新θ来增大$log p(h^*,v)$。 MAP推断作为特征提取器以及一种学习机制被广泛地应用在了深度学习中，主要用于稀疏编码模型中。

**变分推断和学习（Variational Inference and Learning）**：变分学习的核心思想就是我们在一个关于q的有约束的分布族上最大化L。 选择这个分布族时应该考虑到计算$E_qlogp(h,v)$的难易度。 一个典型的方法就是添加分布q如何分解的假设,一种常用的变分学习的方法是加入一些限制使得q是一个因子分布：$q(h∣v)=∏_iq(h_i∣v)$，这被称为**均值场方法（mean field approach）**。 变分方法的优点是我们**不需要为分布q设定一个特定的参数化形式。**我们设定它如何分解，之后通过解决优化问题来找出在这些分解限制下最优的概率分布。 对**离散型潜变量**来说，这意味着我们使用传统的优化技巧来优化描述分布q的有限个变量。 对**连续型潜变量**来说，这意味着我们使用一个被称为变分法的数学工具来解决函数空间上的优化问题。

**离散型潜变量**： 我们定义一个分布q，在最简单的情况中，h是二值的并且我们做了均值场假定，分布q可以根据每一个$h_i$分解。 用一个向量$\hat{h}$来表示参数化分布q，$\hat{h}$的每一个元素都代表一个概率，即$q(h_i=1∣v)=\hat{h}_i$。 然后就可以通过求解$\frac{∂}{∂\hat{h}_i}L=0$，来优化分布q。

**变分的数学基础**：许多机器学习的技巧是基于寻找一个输入向量$θ∈R_n$来最小化函数$J(θ)$，使得它取到最小值。 这个步骤可以利用多元微积分以及线性代数的知识找到满足$∇_θJ(θ)=0$的临界点来完成。 在某些情况下，我们希望能够解一个函数f(x)，比如当我们希望找到一些随机变量的概率密度函数时，借助变分法能够让我们完成这个目标。 **函数f的函数被称为泛函 $J[f]$。** **泛函导数**：即在任意特定的x值，对一个泛函$J[f]$是关于函数f(x)的导数，这也被称为变分导数。 泛函J的关于函数f在点x处的泛函导数被记作$\frac{δ}{δf(x)}J$。 对于可微分函数f(x)以及带有连续导数的可微分函数g(y,x)，有：$\frac{δ}{δf(x)}∫g(f(x),x)dx=\frac{∂}{∂y}g(f(x),x)$。 为了使上述等式更加直观，我们可以把f(x)看作是一个有着无穷不可数多元素的向量，由一个实数向量x作为索引。 这种关系式中描述的泛函导数和对向量$θ∈R_n$的导数相同：$\frac{∂}{∂θ_i}∑_jg(θ_j,j)=\frac{∂}{∂θ_i}g(θ_i,i)$。 

**用变分法求解满足最大熵的分布**：考虑寻找一个定义在$x∈R$上的有最大微分熵的概率密度函数。 熵的定义（连续变量）：$H[p]=−∫p(x)logp(x)dx$。 约束：1.分布p(x)积分值为1；2.方差固定为$σ^2$；3.分布的均值必须为μ。

有：$L[p]=λ_1(∫p(x)dx−1)+λ_2(E[x]−μ)+λ_3(E[(x−μ)^2]−σ^2)+H[p]=∫(λ_1p(x)+λ_2p(x)x+λ_3p(x)(x−μ)^2−p(x)logp(x))dx−λ_1−μλ_2−σ^2λ_3$ 令泛函导数为0：$∀x, \frac{δ}{δp(x)}L=λ_1+λ_2x+λ_3(x−μ)^2−1−logp(x)=0$，解得$p(x)=exp(λ_1+λ_2x+λ_3(x−μ)^2−1)$，令$λ_1=1−logσ\sqrt{2π}, λ_2=0, λ_3=−\frac{1}{2σ^2}$，从而得到$p(x)=N(x;μ,σ^2)$。 这也是当我们不知道真实的分布时总是使用正态分布的一个原因。 因为正态分布拥有最大的熵。

**连续型潜变量**：应用者不需要解决任何变分法的问题，均值场固定点迭代更新有一个通用的方程： 先做均值场近似：$q(h∣v)=∏_iq(h_i∣v)$，
并且对任何的$j≠i$固定$q(h_j∣v)$，只需要满足分布p中任何联合分布变量的概率值不为0，我们就可以通过归一化下面这个未归一的分布:
$\tilde{q}(h_i∣v)=exp(E_{h_{−i}\sim q(h_{−i}∣v)}log\tilde{p}(v,h))$，来得到最优的$q(h_i∣v)$。 

**学得近似推断（Learned Approximate Inference）**：我们可以将优化过程视作将一个输入v映射到一个近似分布$q^∗=\underset{q}{argmax} L(v,q)$的一个函数f。 这样，我们就可以用一个近似函数为$\hat f(v;θ)$的神经网络来近似它。

**醒眠算法（Wake-Sleep Algorithm）**：训练一个可以用v来推断h的模型的主要难在我们没有监督训练集来训练模型。 给定一个v，无法获知一个合适的h。 醒眠算法通过从模型分布中抽取v和h的样本来解决这个问题：如在有向模型中，执行从h开始并在v结束的原始采样v和h；然后这个推断网络可以被训练来执行反向的映射：预测哪一个h产生了当前的v。 再用生物做梦来帮助理解：做梦的作用是训练网络来预测q；清醒的时候更新θ。这解释了动物如何能够保持清醒几个小时（它们清醒的时间越长，L和$logp(v)$之间的差距越大， 但是L仍然是下限）。

****

****

